# The probe phase
There are several ways of analyzing the topology and trajectory pairs, depending on the number of trajectory files per a topology file, the continuity of trjaectory files, organization of files in a directory, and the parallel or sequencial arrangement of the computation powerhorse.


### To-do-list

This notebook is a living documnent. It is an integration of all the past notebooks and scripts written for the *probe* phase in the *PolyPhys* package (or formerly-called *extraction* phase in the decryped *sumrule* package)

BThe list below allows to review the past notebooks and scripts and combine them into this notebook:

- [] Sum-rule: segments: bug test and run on **date:NODATE-YET**
- [] Sum-rule: segments: all test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: bug test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: all test and run on **date:NODATE-YET**
- [] Sum-rule: all_in_one: test and run on **date:NODATE-YET**
- [X] Trans-Foci: segments: bug test and run on **date:20220621**
- [] Trans-Foci: segments: all test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: bug test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: all test and run on **date:NODATE-YET**
- [] Trans-Foci: all_in_one: test and run on **date:NODATE-YET**

### Naming convention:

This is the pattern of file or directory names:

1. **whole** files: whole-group-property_[-measure][-stage][.ext]
2. **ensemble** files: ensemble-group-property_[-measure][-stage][.ext]
3. **ensemble_long** files: ensemble_long-group-property_[-measure][-stage][.ext]
4. **space** files: space-group-property_[-measure][-stage][.ext]
5. **all in one** files: space-group-**species**-**allInOne**-property-_[-measure][-stage][.ext]

[keyword] means that the keyword in the file name is option. [-measure] is a physical measurement such as the auto correlation function (AFC) done on the physical 'property_'.

### Settings for testing and running on a PC.

In [1]:
# settings for testing and running on a PC.
from glob import glob
from polyphys.manage import organizer
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCubic
from polyphys.probe import prober

# HPC Cluster: gnuparallel

## Separated *whole* simulation directories

On a cluster, *whole* simulations are organized into *whole* directories, where each *whole* directory contains all the files for a given *whole* simulation. The **gnuparallel** is used to parallalize the **probe** phase at the **shell** level. For this purpose, all the python modules and scripts are separatedly installed and run on each core. For instance, if 32 cores are available, then the files in 32 *whole* directories are simulatenously installed. However, each *whole* directory may contains multiple toplogy and trajectory pairs. Thus, there is parallelization at the level of *whole* directories, not at the levle of the *segment* or *whole* trajectories inside a *whole* directory. Inside each *whole* directory, a python **main_probe.py** script analyzes the trajectories in a sequencial way.

- trj and all *segments* on a cluster

For each *whole* directory, the following script is executed by means of *gnuparallel*. See these scripts: *probe-1.7-all_trj_segments.py* and *probe-1.7-bug_trj_segments*

### Sum-rule project

#### whole bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober


# analyzing bug files.
geometry = 'biaxial'
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug(
        bug_topo,
        bug_trj,
        geometry,
        lineage,
        save_to = save_to
    )

#### segment bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob('./N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob('./N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRule(
        bug_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug(
            bug_topo, bug_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug(
            bug_topo,
            bug_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./N*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRule(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs:  transverse size

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober


# analyzing bug files.
geometry = 'biaxial'
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_transverse_size(
        bug_topo,
        bug_trj,
        geometry,
        lineage,
        save_to = save_to
    )

#### segment bug trjs: transverse size

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob('./N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob('./N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRule(
        bug_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_transverse_size(
            bug_topo, bug_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_transverse_size(
            bug_topo,
            bug_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_dd

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./N*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRule(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_histdd(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all_histdd(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Trans-Foci project

#### whole bug trjs
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
geometry = 'biaxial'
group='bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug(
        bug_topo, bug_trj, lineage, group, geometry, save_to=save_to
    )

#### segment all trjs

In [ ]:
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFoci(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:

        prober.trans_foci_all(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs: transverse_size
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
geometry = 'biaxial'
group='bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_transverse_size(
        bug_topo, bug_trj, lineage, group, geometry, save_to=save_to
    )

#### segment all trjs: hist_dd

In [ ]:
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFoci(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_histdd(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:

        prober.trans_foci_all_histdd(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

# PC Serial scheme 

There are 4 different types of directories from which only one type can be in a *space* directory.

There are separated **whole** directories in each of which there **all** and **bug** **whole** trajectories; or, there are again separated **whole** directories in each of which there are **all** and **bug** **segment** trjaectories. Below there are two groups of scrips for **serial** and **parallel** runnung schemes.


On a PC, the *whole* directories are located in a master *space-trjs* directory; however, one main python script probes all the *whole* directories in a parallel scheme via Dask. This is different from the *gnuparallel*-based approach in which each *whole* directory has its own copy of the required scripts and a main pytohn script is run to probe that direcotry individually.

## Sumrule projectb

### Cylindrical

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
#macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
bug_pairs = glob(macbookpro_path + '/N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl(
        bug_topo,
        bug_trj,
        lineage,
        continuous=False,
        save_to = save_to
    )

#### segment bug trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob(macbookpro_path + '/N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob(macbookpro_path + '/N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(
        bug_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl(
            bug_topo, bug_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_cyl(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_1d and hist_2d

In [ ]:
# 4 mins for ~4000 particles with one all trj
# macbookmini
parent = "/Users/amirhsi_mini/research_data/trjs"
# macbookpro
#parent = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(parent + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(parent + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRuleCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group,
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs: transverse size

In [ ]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
macmini_path = "/Users/amirhsi_mini/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc4804dt0.005bdump1000adump5000ens6/*.bug*"
#macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
bug_pairs = glob(macbookpro_path + '/N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        continuous=False,
        save_to = save_to
    )

#### segment bug trjs: transverse size

In [ ]:
%%time
# 18 mins for N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7 two bugs
#trjs_db = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
trjs_db = "/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc4804dt0.005bdump1000adump5000ens6"
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob(trjs_db + '/N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob(trjs_db + '/N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(
        bug_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo, bug_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_2d

In [ ]:
%%time
# 10 mins for N2000epsilon5.0r13.0lz259.0sig1.0nc133547dt0.005bdump1000adump5000ens1 2 all trjs
# macbookmini
trjs_db = "/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r13.0lz259.0sig1.0nc133547dt0.005bdump1000adump5000ens1"
# macbookpro
#trjs_db = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(trjs_db + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(trjs_db + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRule(
        all_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
        
    else:
        prober.sum_rule_all_cyl_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

## Trans-Foci project

### Cylindrical

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'bug'
linage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macbookpro_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cyl(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cyl(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment bug trjs: transverse size

In [ ]:
# analyzing bug files.
group = 'bug'
linage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macbookpro_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_foci_bug_cyl_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs: hist_2d

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Cubic

#### whole bug trjs

In [2]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
bug_pairs = glob(macbookpro_path + '/al*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cubic(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.ring-bug is analyzing...



/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/PolyPhys/polyphys/manage/parser.py:1509: UserWarning: It is assumed that 'nc' is the last attribute shortkey in a lineage_name of types: 'ensemble', 'ensemble_long', 'whole', 'segment'.
  warnings.warn(convention_warning, UserWarning)


done.


#### segment all trjs

In [9]:
%%time
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
#macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macbookpro_path + '/al*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macbookpro_path + '/al*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCubic(
        all_trj,
        topo_lineage,
        'cubic',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cubic(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_cubic(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j01.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j02.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j03.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j04.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j05.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j06.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j07.ring-all is analyzing...

done.
Setting the name of analyze file...

al5nl5ml125ns400ac1nc0l62dt0.005bdump2000adump5000ens3.j08.ring-all

#### segment bug trjs: transverse size

In [ ]:
# analyzing bug files.
group = 'bug'
linage = 'whole'
save_to = './'
#macbookpro_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
#macmini_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macmini_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_foci_bug_cubic_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs: hist_2d

In [6]:
%%time
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
#macmini_path = '/Users/amirhsi_mini/research_data/test'
macmini_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/al*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/al*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCubic(
        all_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_cubic_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_cubic_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

IndexError: list index out of range